In [1]:
import os
import re
import numpy as np
from PIL import Image

def sorter(text):
    num = re.findall(r'\d+', text)
    return int(num[0]) if num else 0

data_path = '../../Data/ISBI'
source_img = 'train-volume.tif'
source_lbl = 'train-labels.tif'
source_test_img = 'test-volume.tif'

make_path_list = ['origin/train/image', 'origin/train/label', 'origin/valid/image', 'origin/valid/label', 'origin/test']

for i in make_path_list:
    path = data_path + f'/{i}'
    # 만약 경로에 파일 존재 시, 파일 새로 생성
    if os.path.exists(path):
        print(f'{path} already exists')
    else:
       print(f'{path} is created.')
         
       os.makedirs(path)

print()

tif_img = Image.open(os.path.join(data_path, source_img))
tif_lbl = Image.open(os.path.join(data_path, source_lbl))
tif_test_img = Image.open(os.path.join(data_path, source_test_img))


print(f'[Training data] # of image : {tif_img.n_frames} || # of label : {tif_lbl.n_frames}')
print(f'[Training data] (Hight, Width, "channel") || Image : {tif_img.size} || Label : {tif_lbl.size}')
print()
print(f'[Test data] # of image : {tif_test_img.n_frames} || # of label : {tif_test_img.n_frames}')
print(f'[Test data] (Hight, Width, "channel") || Image : {tif_test_img.size}')

../../Data/ISBI/origin/train/image already exists
../../Data/ISBI/origin/train/label already exists
../../Data/ISBI/origin/valid/image already exists
../../Data/ISBI/origin/valid/label already exists
../../Data/ISBI/origin/test already exists

[Training data] # of image : 30 || # of label : 30
[Training data] (Hight, Width, "channel") || Image : (512, 512) || Label : (512, 512)

[Test data] # of image : 30 || # of label : 30
[Test data] (Hight, Width, "channel") || Image : (512, 512)


In [12]:
# frame에서 랜덤하게 idx를 추출하기 위한 index 값 생성
random_idx = np.arange(tif_img.n_frames)
np.random.shuffle(random_idx)

# train과 valid의 비율이 9:1가 되도록 설정
crit = int(0.9*tif_img.n_frames)
cnt = 1

for i in random_idx:
    tif_img.seek(i)
    tif_lbl.seek(i)
    tif_test_img.seek(i)

    img = tif_img.copy()
    lbl = tif_lbl.copy()
    test_img = tif_test_img.copy()
    
    if cnt <= crit:
        img_save_path = os.path.join(data_path, make_path_list[0])
        lbl_save_path = os.path.join(data_path, make_path_list[1])
    else:
        img_save_path = os.path.join(data_path, make_path_list[2])
        lbl_save_path = os.path.join(data_path, make_path_list[3])

    img.save(img_save_path+f'/image_{cnt-1}.jpg')    
    lbl.save(lbl_save_path+f'/image_{cnt-1}.png')
    test_img.save(data_path+f'/origin/test/image_{cnt-1}.jpg')

    cnt+=1

In [2]:
def img2patches(data_path, patch_size=256):
    data = Image.open(data_path)

    # (left, up, right, down)
    crop_pos = {1: [0,0,256,256], 
                2: [256,0,512,256], 
                3: [0,256,256,512], 
                4: [256,256,512,512]}
    cropped_img1 = data.crop(crop_pos[1])
    cropped_img2 = data.crop(crop_pos[2])
    cropped_img3 = data.crop(crop_pos[3])
    cropped_img4 = data.crop(crop_pos[4])

    cropped_imgs = [cropped_img1, cropped_img2, cropped_img3, cropped_img4]

    return cropped_imgs

In [22]:
origin_train_img_path = '../../Data/ISBI/origin/train/image'
origin_train_lbl_path = '../../Data/ISBI/origin/train/label'

origin_valid_img_path = '../../Data/ISBI/origin/valid/image'
origin_valid_lbl_path = '../../Data/ISBI/origin/valid/label'


modes = ['train', 'valid']


for mode in modes:
    idx = 0

    if mode == 'train':
        img_path = origin_train_img_path
        lbl_path = origin_train_lbl_path
    else:
        img_path = origin_valid_img_path
        lbl_path = origin_valid_lbl_path

    new_img_path = f'../../Data/ISBI/prep/{mode}/image'
    new_lbl_path = f'../../Data/ISBI/prep/{mode}/label'

    paths = [new_img_path, new_lbl_path]

    for path in paths:
        if os.path.exists(path):
            # shutil.rmtree(path)
            continue
        else:
            os.makedirs(path)
    
    
    img_list = sorted(os.listdir(img_path), key=sorter)
    lbl_list = sorted(os.listdir(lbl_path), key=sorter)

    for j in range(len(img_list)):
        img = os.path.join(img_path, img_list[j])
        lbl = os.path.join(lbl_path, lbl_list[j])

        img_patches = img2patches(img, 256)
        lbl_patches = img2patches(lbl, 256)

        for k in range(len(img_patches)):
            input_tile = img_patches[k]
            new_mask = lbl_patches[k]

            input_tile.save(new_img_path + f'/image_{idx}.jpg')
            new_mask.save(new_lbl_path + f'/image_{idx}.png')
            idx += 1 